<a href="https://colab.research.google.com/github/Dzamora-210/BERT-Sentiment-Model/blob/main/Copy_of_Sentiment_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LOAD NECESSITIES

In [ ]:
!pip install transformers
!pip install scipy
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install pandas 
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 22.6 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 6.6 MB 40.9 MB/s 
     |████████████████████████████████| 596 kB 41.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.4 MB/s eta 0:14:05tcmalloc: large alloc 1147494400 bytes == 0x3a2e6000 @  0x7fd787b0f615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3

IMPORT LIBRARIES/PACKAGES 

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification 
import torch
import pandas as pd
import numpy as np


UPLOAD EXPORTED COMMENTS AS CSV

In [ ]:
df = pd.read_csv('/content/Cisco_Webex EMEA_grading_datasheet - comments_grading.csv')
df = df[['Comments']]
df.head(10)

,Comments
0,🥪🔔
1,কি
2,☎️☎️
3,How
4,Awesome
5,"Am working on it,today's night 😌"
6,Super 😗
7,very good
8,So sweet
9,Awesome


LOAD MODEL

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

ENCODE/TOKENIZE COMMENTS

In [ ]:
def sentiment_score(df):
    tokens = tokenizer.encode(df, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))

In [ ]:
df = df.dropna()

VIZUAL

In [ ]:
df['Comments'].iloc[5]

"Am working on it,today's night 😌"

In [ ]:
sentiment_score(df['Comments'].iloc[5])

4

APPLY SCOREING TO ALL COMMENTS/CLASSIFY GRADING

In [ ]:
df['score'] = df['Comments'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
def getclassification(label): 

  if label < 2:
    return 'Negative'
  if label == 2: 
    return 'Neutral'
  elif label > 2:
    return 'Positive'

In [ ]:
df['sentiment'] = df['score'].apply(getclassification)

In [ ]:
df

,Comments,score,sentiment
0,🥪🔔,4,Positive
1,কি,2,Neutral
2,☎️☎️,4,Positive
3,How,4,Positive
4,Awesome,4,Positive
...,...,...,...
205,https://www.youtube.com/watch?v=dnFVYewJZgA\n ...,0,Negative
206,Solo per chi ha lavorato tramite agenzie del l...,0,Negative
207,Very nice,4,Positive
208,Info,3,Positive


SENTIMENT BREAKDOWN

In [ ]:
positive = df[df['sentiment'] == 'Positive'] 
print(str(positive.shape[0]/(df.shape[0])*100)+ "% positive sentiment")
pos = positive.shape[0]/df.shape[0]*100

77.61904761904762% positive sentiment


In [ ]:
neutral = df[df['sentiment'] == 'Neutral'] 
print(str(neutral.shape[0]/(df.shape[0])*100)+ "% neutral sentiment")
pos = neutral.shape[0]/df.shape[0]*100

7.142857142857142% neutral sentiment


In [ ]:
negative = df[df['sentiment'] == 'Negative'] 
print(str(negative.shape[0]/(df.shape[0])*100)+ "% negative sentiment")
pos = negative.shape[0]/df.shape[0]*100

15.238095238095239% negative sentiment


EXPORT GRADED COMMENTS AS EXCEL

In [ ]:
df.to_excel (r'C:Sentiment Analysis.xlsx', index = False, header=True)

QA MODEL GRADING AND SEND CORRECTED GRADINGS TO DANTE